# Spatiotemporal Graph Neural Network Example
This demonstrates a simple GNN over a dynamic graph using PyTorch Geometric.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

# Create toy dynamic graph (fixed edges, varying features)
edge_index = torch.tensor([[0,1,1,2],
                           [1,0,2,1]], dtype=torch.long)

T = 20
features = []
labels = []
for t in range(T):
    x = torch.randn(3, 4) + t*0.1  # 3 nodes, 4 features
    y = torch.tensor([t%2, (t+1)%2, t%2], dtype=torch.long)
    features.append(x)
    labels.append(y)

class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

model = GCN(4, 8, 2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(30):
    loss_epoch = 0
    for t in range(T):
        x, y = features[t], labels[t]
        out = model(x, edge_index)
        loss = loss_fn(out, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_epoch += loss.item()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss {loss_epoch:.4f}")